In [20]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded=files.upload()

Saving Restaurant_Reviews.tsv to Restaurant_Reviews (1).tsv


In [21]:
data=pd.read_csv('Restaurant_Reviews.tsv',delimiter='\t')
data

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


**CLEANING TEXT**

In [22]:

from os import R_OK
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
for i in range(0,1000):
  Review=re.sub('[^a-zA-Z]',' ',data['Review'][i])
  Review=Review.lower()
  Review=Review.split()
  ps=PorterStemmer()
  Review=[ps.stem(word)for word in Review
          if not word in set(stopwords.words('english'))]
  Review=' '.join(Review)
  corpus.append(Review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**CREATING A BAG OF WORDS MODEL**

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
x=cv.fit_transform(corpus).toarray()
y=data.iloc[:,1].values

**SPLITING A DATASET**

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

**training the SVM model on the Training Data**

In [25]:
from sklearn.svm import SVC
Classifier=SVC(kernel='linear',random_state=0)
Classifier.fit(x_train,y_train)

SVC(kernel='linear', random_state=0)

**predicting the test results**

In [26]:
y_pred=Classifier.predict(x_test)
y_pred
acc=round(Classifier.score(x_test,y_test)*100,2)
print(str(acc)+'%')

78.4%


**predicting the sample review**

In [27]:
def predict_review(sample_review):
  Sample_review=re.sub(pattern='[^a-zA-Z]',repl=' ',string=sample_review)
  sample_review=sample_review.lower()
  sample_review_words=sample_review.split()
  sample_review_words=[word for word in sample_review_words if not word in set(stopwords.words('english'))]
  ps=PorterStemmer()
  final_review=[ps.stem(word)for word in sample_review_words]
  final_review=' '.join(final_review)
  temp=cv.transform([final_review]).toarray()
  return Classifier.predict(temp)



In [28]:
sample_review="its very worst"
if predict_review(sample_review):
  print("positive review")
else:
  print("negative review")


negative review


In [29]:
sample_review="its very good"
if predict_review(sample_review):
  print("positive review")
else:
  print("negative review")


positive review
